# trade blotter

## Basic FastAPI app structure

In [3]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/trades")
async def get_trades():
    # Replace with broker API logic
    return {"message": "Trades data will be here"}


## python tt backend

In [ ]:
from flask import Flask, jsonify
import yaml
import requests
import json
import pandas as pd
import platform
from datetime import datetime
import sqlite3

app = Flask(__name__)

ps = platform.system()

if ps == "Darwin":
    creds_file = "creds.yaml"
if ps == "Linux":
    creds_file = "/home/ec2-user/tt/creds.yaml"

# Database connection
conn = sqlite3.connect('nav.db')  # This will create the database file if it doesn't exist
try:
    nav_df = pd.read_sql('SELECT * FROM nav', conn)
except:
    nav_df = pd.DataFrame(columns=['timestamp', 'NAV'])

# Load credentials
with open(creds_file, "r") as file:
    data = yaml.safe_load(file)
    pw = data.get("pw")[0]
    user = data.get("user")[0]

# Tastytrade API session URL
SESSION_URL = 'https://api.tastyworks.com/sessions'
TRANSACTIONS_URL = 'https://api.tastyworks.com/accounts/5WY49300/transactions'

def get_session_token():
    data = {
        "login": user,
        "password": pw,
        "remember-me": True
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(SESSION_URL, data=json.dumps(data), headers=headers)
    response.raise_for_status()
    session_data = response.json()
    return session_data['data']['session-token']

def get_tastytrade_transactions(session_token):
    headers = {
        'Authorization': session_token,
        'Content-Type': 'application/json'
    }
    response = requests.get(TRANSACTIONS_URL, headers=headers)
    response.raise_for_status()
    transactions_data = response.json()
    return pd.DataFrame(transactions_data['data']['items'])

def close_session(session_token):
    headers = {
        'Authorization': session_token,
        'Content-Type': 'application/json'
    }
    response = requests.delete(SESSION_URL, headers=headers)
    response.raise_for_status()

@app.route('/trades', methods=['GET'])
def fetch_trades():
    try:
        session_token = get_session_token()
        transactions_df = get_tastytrade_transactions(session_token)
        close_session(session_token)
        print('t')
        print(transactions_df)
        print(jsonify(transactions_df.to_dict(orient='records')))
        return jsonify(transactions_df.to_dict(orient='records'))
    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    print('pre main')
    app.run(port=5000, debug=True)


pre main
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 1042, in launch_instance
    app.initialize(argv)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 113, in inner
    return method(app, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/pytho

SystemExit: 1

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

In [5]:
import yaml
import requests
import json
import pandas as pd
import platform
from datetime import datetime
import sqlite3

ps = platform.system()

if ps == "Darwin":
    creds_file = "creds.yaml"
if ps == "Linux":
    creds_file = "/home/ec2-user/tt/creds.yaml"

# Load credentials
with open(creds_file, "r") as file:
    data = yaml.safe_load(file)
    pw = data.get("pw")[0]
    user = data.get("user")[0]

# Tastytrade API session URL
SESSION_URL = 'https://api.tastyworks.com/sessions'
TRANSACTIONS_URL = 'https://api.tastyworks.com/accounts/5WY49300/transactions'

def get_session_token():
    data = {
        "login": user,
        "password": pw,
        "remember-me": True
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(SESSION_URL, data=json.dumps(data), headers=headers)
    response.raise_for_status()
    session_data = response.json()
    return session_data['data']['session-token']

def get_tastytrade_transactions(session_token):
    headers = {
        'Authorization': session_token,
        'Content-Type': 'application/json'
    }
    response = requests.get(TRANSACTIONS_URL, headers=headers)
    response.raise_for_status()
    transactions_data = response.json()
    return pd.DataFrame(transactions_data['data']['items'])

def close_session(session_token):
    headers = {
        'Authorization': session_token,
        'Content-Type': 'application/json'
    }
    response = requests.delete(SESSION_URL, headers=headers)
    response.raise_for_status()


def fetch_trades():
    try:
        session_token = get_session_token()
        transactions_df = get_tastytrade_transactions(session_token)
        close_session(session_token)
        print('t')
        print(transactions_df)
        transactions_df.to_clipboard(index=False)
        print(jsonify(transactions_df.to_dict(orient='records')))
        return jsonify(transactions_df.to_dict(orient='records'))
    except:
        print("Error fetching trades")

if __name__ == "__main__":
    print('pre main')
    fetch_trades()


pre main
t
            id account-number         action clearing-fees  \
0    337302016       5WY49300   Sell to Open         0.008   
1    337282358       5WY49300   Sell to Open         0.011   
2    337262724       5WY49300   Sell to Open         0.017   
3    337240673       5WY49300   Sell to Open         0.003   
4    337213693       5WY49300   Sell to Open         0.023   
..         ...            ...            ...           ...   
245  330013899       5WY49300  Sell to Close           NaN   
246  330013893       5WY49300   Buy to Close           NaN   
247  330013114       5WY49300   Buy to Close           NaN   
248  329874745       5WY49300   Sell to Open           0.2   
249  329874742       5WY49300    Buy to Open           0.1   

    clearing-fees-effect commission commission-effect currency  \
0                  Debit        0.0              None      USD   
1                  Debit        0.0              None      USD   
2                  Debit        0.0           